In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Passaggio 1: Definire i modelli Pydantic per output strutturati

Questi modelli definiscono lo **schema** che gli agenti restituiranno. Utilizzare `response_format` con Pydantic garantisce:
- ✅ Estrazione dei dati sicura per tipo
- ✅ Validazione automatica
- ✅ Nessun errore di analisi da risposte in testo libero
- ✅ Facile instradamento condizionale basato sui campi


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Passaggio 2: Creare lo Strumento di Prenotazione Hotel

Questo strumento sarà utilizzato dall'**availability_agent** per verificare la disponibilità delle camere. Usiamo il decoratore `@ai_function` per:
- Convertire una funzione Python in uno strumento richiamabile dall'AI
- Generare automaticamente lo schema JSON per l'LLM
- Gestire la validazione dei parametri
- Abilitare l'invocazione automatica da parte degli agenti

Per questa demo:
- **Stoccolma, Seattle, Tokyo, Londra, Amsterdam** → Camere disponibili ✅
- **Tutte le altre città** → Nessuna camera disponibile ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Passaggio 3: Definire le Funzioni di Condizione per il Routing

Queste funzioni esaminano la risposta dell'agente e determinano quale percorso seguire nel flusso di lavoro.

**Schema Chiave:**
1. Verificare se il messaggio è `AgentExecutorResponse`
2. Analizzare l'output strutturato (modello Pydantic)
3. Restituire `True` o `False` per controllare il routing

Il flusso di lavoro valuterà queste condizioni sui **bordi** per decidere quale executor invocare successivamente.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Passaggio 4: Creare un Executor di Visualizzazione Personalizzato

Gli executor sono componenti del workflow che eseguono trasformazioni o effetti collaterali. Utilizziamo il decoratore `@executor` per creare un executor personalizzato che visualizza il risultato finale.

**Concetti Chiave:**
- `@executor(id="...")` - Registra una funzione come executor del workflow
- `WorkflowContext[Never, str]` - Suggerimenti di tipo per input/output
- `ctx.yield_output(...)` - Produce il risultato finale del workflow


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Passaggio 5: Carica le variabili d'ambiente

Configura il client LLM. Questo esempio funziona con:
- **Modelli GitHub** (Livello gratuito con token GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Passaggio 6: Creare Agenti AI con Output Strutturati

Creiamo **tre agenti specializzati**, ciascuno avvolto in un `AgentExecutor`:

1. **availability_agent** - Verifica la disponibilità degli hotel utilizzando lo strumento
2. **alternative_agent** - Suggerisce città alternative (quando non ci sono camere disponibili)
3. **booking_agent** - Incoraggia la prenotazione (quando ci sono camere disponibili)

**Caratteristiche principali:**
- `tools=[hotel_booking]` - Fornisce lo strumento all'agente
- `response_format=PydanticModel` - Impone un output JSON strutturato
- `AgentExecutor(..., id="...")` - Avvolge l'agente per l'uso nel flusso di lavoro


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Passaggio 7: Costruire il Workflow con Edge Condizionali

Ora utilizziamo `WorkflowBuilder` per costruire il grafo con il routing condizionale:

**Struttura del Workflow:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Metodi Chiave:**
- `.set_start_executor(...)` - Imposta il punto di ingresso
- `.add_edge(from, to, condition=...)` - Aggiunge un edge condizionale
- `.build()` - Finalizza il workflow


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Passaggio 8: Esegui il Caso di Test 1 - Città SENZA Disponibilità (Parigi)

Testiamo il percorso **senza disponibilità** richiedendo hotel a Parigi (che non ha camere nella nostra simulazione).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Passaggio 9: Esegui il Caso di Test 2 - Città CON Disponibilità (Stoccolma)

Ora testiamo il percorso di **disponibilità** richiedendo hotel a Stoccolma (che ha camere nella nostra simulazione).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Punti Chiave e Prossimi Passi

### ✅ Cosa Hai Imparato:

1. **Pattern WorkflowBuilder**
   - Usa `.set_start_executor()` per definire il punto di ingresso
   - Usa `.add_edge(from, to, condition=...)` per il routing condizionale
   - Chiama `.build()` per finalizzare il workflow

2. **Routing Condizionale**
   - Le funzioni condizionali analizzano `AgentExecutorResponse`
   - Interpretano output strutturati per prendere decisioni di routing
   - Restituiscono `True` per attivare un collegamento, `False` per saltarlo

3. **Integrazione degli Strumenti**
   - Usa `@ai_function` per convertire funzioni Python in strumenti AI
   - Gli agenti chiamano automaticamente gli strumenti quando necessario
   - Gli strumenti restituiscono JSON che gli agenti possono interpretare

4. **Output Strutturati**
   - Usa modelli Pydantic per estrazione dati con tipi sicuri
   - Imposta `response_format=MyModel` quando crei agenti
   - Analizza le risposte con `Model.model_validate_json()`

5. **Executor Personalizzati**
   - Usa `@executor(id="...")` per creare componenti del workflow
   - Gli executor possono trasformare dati o eseguire effetti collaterali
   - Usa `ctx.yield_output()` per produrre risultati del workflow

### 🚀 Applicazioni Reali:

- **Prenotazione Viaggi**: Controlla disponibilità, suggerisci alternative, confronta opzioni
- **Servizio Clienti**: Instrada in base al tipo di problema, sentiment, priorità
- **E-commerce**: Controlla inventario, suggerisci alternative, processa ordini
- **Moderazione Contenuti**: Instrada in base a punteggi di tossicità, segnalazioni utenti
- **Workflow di Approvazione**: Instrada in base a importo, ruolo utente, livello di rischio
- **Elaborazione Multi-fase**: Instrada in base a qualità e completezza dei dati

### 📚 Prossimi Passi:

- Aggiungi condizioni più complesse (criteri multipli)
- Implementa cicli con gestione dello stato del workflow
- Aggiungi sotto-workflow per componenti riutilizzabili
- Integra con API reali (prenotazione hotel, sistemi di inventario)
- Aggiungi gestione degli errori e percorsi di fallback
- Visualizza i workflow con gli strumenti di visualizzazione integrati



---

**Clausola di esclusione della responsabilità**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche potrebbero contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si consiglia una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
